### Homework 2.1.1 - 1D Wave

Solve the wave equation

$$\frac{{{\partial ^2}u}}{{\partial {t^2}}} = c^2 \frac{{{\partial ^2}u}}{{\partial {x^2}}}$$

subjected to the **feeding wave boundary condition** at $x = 0$,

$$u(0, t) = 2\sin(\omega t)$$

and the boundary condition on the other end, $x = L$ is an **open boundary condition**.

where $L = 1$ and the initial condition at $t=0$ is:

$$u(x, 0) = 0$$

You choose appropreate values of **the speed ($c$)** and **the frequency ($\omega$)** to make a nice time-animated graph video clip.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange, tqdm
import imageio
import os

In [4]:
deltaT = 0.0001
deltaX = 0.01

tMin = 0
tMax = 10

xMin = 0
xMax = 1

Nt = int(tMax/deltaT)  # Temporal steps
Nx = int(xMax/deltaX)  # Spatial steps

t = np.linspace(tMin, tMax, num=Nt, endpoint=True)
x = np.linspace(xMin, xMax, num=Nx, endpoint=True)

# Compute Secondary variables
c = 1
omega = 20
h = xMax / Nx
eps = (deltaT*c / h)**2
# Error propagates if eps > 1. Make sure dT,dX,c do not produce eps > 1
print('Eps:', eps)

U = np.zeros((Nx, Nt), dtype=np.longdouble)

# Compute U(j=0). We will compute U(j>0) later in the iteration below.
U[0][1] = 2*np.sin(omega*1*deltaT)

# Iterate to the matrix u
for j in trange(1, Nt-1):
    U[0][j+1] = 2*np.sin(omega*(j+1)*deltaT)
    U[-1][j+1] = U[-1][j] - c*(U[-1][j] - U[-2][j])

    for i in range(1, Nx-1):
        U_ip1 = U[i+1][j]
        U_im1 = U[i-1][j]
        U_ij = U[i][j]
        U_jm1 = U[i][j-1]

        U[i][j+1] = eps*(U_ip1 + U_im1) + 2.*(1. - eps)*U_ij - U_jm1

frames = []
for i in trange(0, Nt, 100):
    plt.plot(x, U[:, i])
    plt.ylim(-5, 5)
    plt.grid()

    # make frame
    frame = f'{i}.png'
    frames.append(frame)
    # save frame
    plt.savefig(frame)
    plt.close()

with imageio.get_writer('feed-wave-2-1-1.gif', mode='I') as writer:
    for frame in frames:
        image = imageio.imread(frame)
        writer.append_data(image)

# Remove files
for filename in set(frames):
    os.remove(filename)
print("Done!")

Eps: 0.0001


  0%|          | 0/99998 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Done!


In [5]:
%%HTML
<img src="feed-wave-2-1-1-ans.gif">

___
### Homework 2.1.2 - 1D Wave with damping

Solve the wave equation

$$\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2} - k \frac{\partial u}{\partial t}$$

subjected to the **feeding wave boundary condition** at $x = 0$,

$$u(0, t) = 2\sin(\omega t)$$

and the boundary condition on the other end, $x = L$ is an **open boundary condition**.

where $L = 1$ and the initial condition at $t=0$ is:

$$u(x, 0) = 0$$

You choose appropreate values of **the speed ($c$)**, **the frequency ($\omega$)**, and **the damping factor ($k$)** to make a nice time-animated graph video clip.

$$
\begin{align}
\frac{\partial^2 P}{\partial t^2} &= c^2 \frac{\partial^2 P}{\partial x^2} - k \frac{\partial P}{\partial t} \\
\frac{P_{j+1} + 2P + P_{j-1}}{dt^2} &=
\frac{c^2(P_{i+1}+2P+P_{i-1})}{h^2} -
\frac{k(P_{j+1}-P_{j-1})}{2dt} \\
\frac{P_{j+1} + 2P + P_{j-1}}{dt^2} &=
\frac{2dt\cdot c^2(P_{i+1}+2P+P_{i-1})-h^2 \cdot k(P_{j+1}-P_{j-1}}{2dt \cdot h^2} \\
\frac{P_{j+1} + 2P + P_{j-1}}{dt} &=
\frac{2dt\cdot c^2(P_{i+1}+2P+P_{i-1})-h^2 \cdot k(P_{j+1}-P_{j-1}}{2 \cdot h^2} \\
2h^2P_{j+1} + 4h^2P + 2h^2P_{j-1} &=
2dt^2c^2P_{i+1}+4dt^2c^2P+2dt^2c^2P_{i-1} - kdth^2P_{j+1}+kdth^2P_{j-1} \\
(2h^2+kdth^2)P_{j+1} &=
2dt^2c^2(P_{i+1}+P_{i-1})+(dt^2c^2-h^2)4P+(kdt-2)h^2P_{j-1} \\
P_{j+1} &= \frac{2dt^2c^2(P_{i+1}+P_{i-1})+(dt^2c^2-h^2)4P+(kdt-2)h^2P_{j-1}}{2h^2+kdth^2}
\end{align}
$$

Hence our $U_{i, j+1}$ is:
$$
U_{i,j+1} = \frac{2\delta^2c^2(U_{i+1,j}+U_{i-1,j})+(\delta^2c^2-h^2)4U_{i,j} +
(k\delta-2)h^2U_{i,j-1}}{2h^2+k\delta h^2}
$$

In [6]:
deltaT = 0.0001
deltaX = 0.01

tMin = 0
tMax = 10

xMin = 0
xMax = 1

Nt = int(tMax/deltaT)  # Temporal steps
Nx = int(xMax/deltaX)  # Spatial steps

t = np.linspace(tMin, tMax, num=Nt, endpoint=True)
x = np.linspace(xMin, xMax, num=Nx, endpoint=True)

# Compute Secondary variables
c = 1
k = 2
omega = 10
h = xMax / Nx
coeff1 = 2*(deltaT**2)*(c**2)
coeff2 = (deltaT**2)*(c**2) - h**2
coeff3 = k*deltaT - 2
denorm = 2*(h**2) + k*deltaT*(h**2)


U = np.zeros((Nx, Nt), dtype=np.longdouble)

# Compute U at temporal 1 and Spatial 0
U[0][1] = 2*np.sin(omega*1*deltaT)

# Iterate to the matrix u
for j in trange(1, Nt-1):
    U[0][j+1] = 2*np.sin(omega*(j+1)*deltaT)
    U[-1][j+1] = U[-1][j] - c*(U[-1][j] - U[-2][j])

    for i in range(1, Nx-1):
        U_ip1 = U[i+1][j]
        U_im1 = U[i-1][j]
        U_ij = U[i][j]
        U_jm1 = U[i][j-1]

        first = coeff1 * (U_ip1 - U_im1)
        middle = coeff2 * 4 * U_ij
        last = coeff3 * h**2 * U_jm1
        norm = first + middle + last

        U[i][j+1] = norm / denorm

frames = []
for i in trange(0, Nt, 50):
    plt.plot(x, U[:, i])
    plt.grid()

    # make frame
    frame = f'{i}.png'
    frames.append(frame)
    # save frame
    plt.savefig(frame)
    plt.close()

with imageio.get_writer('feed-wave-2-1-2.gif', mode='I') as writer:
    for frame in frames:
        image = imageio.imread(frame)
        writer.append_data(image)

# Remove files
for filename in set(frames):
    os.remove(filename)
print("Done!")

  0%|          | 0/99998 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

Done!


In [7]:
%%HTML
<img src="feed-wave-2-1-2-ans.gif">